In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.font_manager import FontProperties

# matplotlib.style.use('ggplot')
import matplotlib
# matplotlib.style.use('ggplot')
#matplotlib.style.use('default')
from numpy import *
import numpy as np
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (8, 6)
import subprocess,shlex,sys,select,re,StringIO
def readHive(sql):
    text = ''
    err = ''
    cmd = shlex.split('hive -e '+ '"set mapreduce.job.queuename=zampda_job;set hive.cli.print.header=true;'+sql+'"')
    print >>sys.stderr, cmd
    sys.stderr.flush()
    proc = subprocess.Popen(cmd, bufsize=0, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    while proc.poll() is None:
        ins, outs, errs = select.select([proc.stdout, proc.stderr], [], [])
        if proc.stdout in ins:
            text += proc.stdout.read()
        if proc.stderr in ins:
            c = proc.stderr.read(1)
            if c == '\n':
                print >>sys.stderr, err
                sys.stderr.flush()
                err = ''
            else:
                err += c
    text += proc.stdout.read()
    print >>sys.stderr, proc.stderr.read()
    sys.stderr.flush()
    header, content = text.split('\n',1)
    header = re.sub('\w*\.','', header)
    return pd.read_table(StringIO.StringIO(header+'\n'+content), error_bad_lines = False)

def f(x):
    x=str(x)
    tmp=x.split("//")
    if len(tmp)<=1:
        return x
    return '.'.join(x.strip().split('//')[1].split('/')[0].split('.')[-2:])

Populating the interactive namespace from numpy and matplotlib


In [3]:
clk=readHive("""select * from fraud_free.dclkv2 where year=2016 and month=8 and day=17""")
ld=readHive("""select * from etl_output.landing where year=2016 and month=8 and day=17 and request_id is not null""")

['hive', '-e', 'set mapreduce.job.queuename=zampda_job;set hive.cli.print.header=true;select * from fraud_free.dclkv2 where year=2016 and month=8 and day=17']

Logging initialized using configuration in file:/var/bh/lib/apache-hive-1.2.1-bin/conf/hive-log4j.properties
OK
 Fetched: 552107 row(s)

/home/jay/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
['hive', '-e', 'set mapreduce.job.queuename=zampda_job;set hive.cli.print.header=true;select * from etl_output.landing where year=2016 and month=8 and day=17 and request_id is not null']

Logging initialized using configuration in file:/var/bh/lib/apache-hive-1.2.1-bin/conf/hive-log4j.properties
OK
conds, Fetched: 99280 row(s)



In [17]:
j=pd.merge(clk,ld,on='request_id',how='left')
print len(j),len(j.query("fraud_flag==fraud_flag"))

976641 520295


In [18]:
print len(clk),len(clk.query("fraud_flag==fraud_flag"))

552107 141207


In [20]:
clk['domain']=clk['url'].apply(f)
total=j['adslot_id'].value_counts().reset_index()
total.rename(columns={'adslot_id':'all'},inplace=True)
total_clean=j.query("fraud_flag==fraud_flag")['adslot_id'].value_counts().reset_index()
total_clean.rename(columns={'adslot_id':'clean'},inplace=True)
# total_ld=j.query("year_y==year_y")['adslot_id'].value_counts().reset_index()
# total_ld.rename(columns={'adslot_id':'ld'},inplace=True)
first=pd.merge(total,total_clean,on='index',how='inner')
first['cleanr']=first['clean']/first['all']
# second=pd.merge(first,total_ld,on='index',how='inner')
# second['cleanr']=second['clean']/second['all']
# second['landingr']=second['ld']/second['all']
third=pd.merge(first,clk[['adslot_id','domain','vendor_id']].drop_duplicates(),left_on='index',right_on='adslot_id',how='inner')

In [22]:
clk.query("fraud_flag==fraud_flag")['fraud_flag'].value_counts()

1.0    62391
0.0    52032
5.0    24611
2.0     2173
Name: fraud_flag, dtype: int64

In [14]:


# third.query('cleanr<=0.3 &vendor_id==10& domain in (\'youku.com\',\'tudou.com\',\'ynet.com\',\'miercn.com\',\'mianhuatang.la\',\'xinjunshi.com\',\'9ku.com\',\'le.com\',\'y2002.com\',\'eastday.com\',\'faloo.com\',\'fxingw.com\',\'cntv.cn\',\'wasu.cn\',\'555zw.com\',\'xiaoxiaoshuwu.com\',\'manhuatai.com\',\'d8qu.com\')')[['index','adslot_id_x','cleanr','landingr','domain','vendor_id']]

In [ ]:
third